In [1]:
import numpy as np
from qutip import *
from sympy.physics.quantum.cg import CG
from sympy.physics.wigner import wigner_6j, wigner_3j
from tqdm import tqdm
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy import linalg as LA
from scipy.optimize import minimize_scalar
from scipy.sparse.linalg import eigs

In [2]:
I = 7
S = 1
Je = 0
Jg = 1
Lg = 2
F = [i for i in range(int(I-Jg),int(I + Jg + 1),1)]
F.insert(0,int(I+Je))
F 

[7, 6, 7, 8]

In [3]:
N = [2*F[i] + 1 for i in range(len(F))]
N = sum(N)
N

60

In [4]:
#Zeeman stuff


muB = 9.3e-24 #Bohr magneton J T^-1
hbar = 1.054e-34 # J s

muB = muB/hbar #hbar = 1
muB = muB/(2*np.pi)
muB = muB/1e6  # so it's in 1/2π (MHz T^-1)

# 1Gauss is 0.0001 T
B = 4e-4
print(muB*B) 



#Lande g factor for F
me = 9.1093837015e-31 
mp = 1.67262192369e-27 

mI = 3.169 #nuclear magnetic moment of 176Lu in units of μN
gI = mI/I

gJg = 1 + (Jg*(Jg +1) + S*(S + 1) - Lg*(Lg + 1))/(2*Jg*(Jg + 1))
print(gJg)

#gF = [1,0,1,2]
gF=[]
gF.append(-gI*me/mp)

for i in range(3):
    gF.append(gJg*(F[i +1]*(F[i+1] + 1) + Jg*(Jg+1) - I*(I+1))/(2*F[i+1]*(F[i +1] + 1)))
    
print(gF)

5.617233285596307
0.5
[-0.0002465559058692723, -0.07142857142857142, 0.008928571428571428, 0.0625]


In [5]:
for i in range(len(F)):
    print(gF[i]*muB*B) #(1/2π MHz)

-0.0013849620412092264
-0.4012309489711648
0.0501538686213956
0.3510770803497692


In [6]:
def base(F,i,mF):
    #numbering states from 0 to N-1 starting from -mF to mF
    # 0 is |F',-mF'>
    if i==0:
        b = basis(N,mF+F[i])
    elif i==1:
        b = basis(N,mF + F[i] + 2*F[0] + 1)
    elif i==2:
        b = basis(N,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1)
    else:
        b = basis(N,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1 + 2*F[2] +1)
    return b    

## Dissipation

In [7]:
GammaJgJe = 2.44745 #1/2π (MHz) 3D1 to 3P0
#GammaJgJe = 2.5

In [8]:
def GammaFgFe(F,ig,Je,Jg,I,GammaJgJe):
    return float((2*Je + 1)*(2*F[0] + 1)*wigner_6j(Je,F[0],I,F[ig],Jg,1)**2)*GammaJgJe
    #return GammaJgJe

In [9]:
GammaFgFe(F,1,Je,Jg,I,GammaJgJe) + GammaFgFe(F,2,Je,Jg,I,GammaJgJe) + GammaFgFe(F,3,Je,Jg,I,GammaJgJe)

2.44745

In [10]:
def cg(F,ig,mFg,ie,mFe,q):
    return float(CG(F[ig],mFg,1,q,F[ie],mFe).doit())
    #return 1

In [13]:
#checking our normalization convention
for ig in range(1,len(F)):
    we=0
    for mfg in range(-F[ig],F[ig] + 1):
        for mfe in range(-F[0],F[0]+1):
            for q in [-1,0,1]:
                we+=1/(2*F[0] + 1)*cg(F,ig,mfg,0,mfe,q)**2
                
    print(we)

1.0
1.0
0.9999999999999999


In [14]:
#for ig in range(1,len(F)):
#    for mFe in range(-F[0],F[0]+1):
#        for mFg in range(-F[ig],F[ig]+1):
#            for q in qs:
#                print((1/np.sqrt(2*F[0] + 1)*cg(F,ig,mFg,0,mFe,q))**-2)

In [15]:
#qs = [-1,0,1]
#c_ops = []

#for ig in range(1,len(F)):
#    for q in qs:
#        cops = GammaFgFe(F,ig,Je,Jg,I,GammaJgJe)*( spre(Ddagger(F,ig,q).dag())*spost(Ddagger(F,ig,q)) 
#           - 0.5*(spre(Ddagger(F,ig,q)*Ddagger(F,ig,q).dag()) 
#           + spost(Ddagger(F,ig,q)*Ddagger(F,ig,q).dag())) ) 
        
#        #for each decay with polarization q from F' to F we write a different superoperator
#        c_ops.append(cops)   

In [16]:
#single collapse operator for each transtion
c_ops = []
qs = [-1,0,1]

for ig in range(1,len(F)):
    for mfg in range(-F[ig],F[ig] + 1):
        for mfe in range(-F[0], F[0] + 1):
            for q in qs:
                if cg(F,ig,mfg,0,mfe,q) != 0:
                    cops =np.sqrt(1/(2*F[0] + 1))*cg(F,ig,mfg,0,mfe,q)*np.sqrt(GammaFgFe(F,ig,Je,Jg,I,GammaJgJe))*base(F,ig,mfg)*base(F,0,mfe).dag()
                    c_ops.append(cops.to("CSR"))
                else:
                    continue

In [17]:
#check
#sum=0
#qs = [-1,0,1]

#for ig in range(1,len(F)):
#    for mfg in range(-F[ig],F[ig] + 1):
#        for mfe in range(-F[0], F[0] + 1):
#            for q in qs:
#                sum+= (np.sqrt(1/(2*F[0] + 1))*cg(F,ig,mfg,0,mfe,q)*np.sqrt(GammaFgFe(F,ig,Je,Jg,I,GammaJgJe)))**2
#sum

## Steady state 

In [18]:
def H_I(F,Omega_p):
    HI=0*basis(N,0)*basis(N,0).dag()
    for mFe in range(-F[0],F[0] + 1):
        for ig in range(1,len(F)):
            for mFg in range(-F[ig],F[ig]+1):
                for q in range(-1,2):
                    HI += cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*base(F,0,mFe)*base(F,ig,mFg).dag()
                
    return 1/np.sqrt(2*F[0]+1)*(HI + HI.dag()) 

In [19]:
def H_0(F,Delta):
    H0 = 0*basis(N,0)*basis(N,0).dag()
    for l in range(len(F)):
        for mF in range(-F[l],F[l]+1):
            H0 += (Delta[l] + gF[l]*muB*mF*B)*base(F,l,mF)*base(F,l,mF).dag() 
    return H0

### σ+ and σ- for all transitions

In [20]:
#Rabi frequencies Ω_F,F' #1/2π (MHz)
omega = [73,19,73]

#turn on/off polarization
Omega_p= np.zeros((3,3)) # (ig-1, q + 1) transition from F state to F' with polarization q

Omega_p[0,0] = Omega_p[0,2] = omega[0]       #σ+ and σ- for F=F[1] to F'= F[0]
Omega_p[1,0] = Omega_p[1,2] = omega[1]      #σ+ and σ- for F=F[2] to F'=F[0]
Omega_p[2,0] = Omega_p[2,2] = omega[2]     #σ+ and σ- for F=F[3] to F'=F[0]

In [21]:
B = 4e-4
print(muB*B) 
Delta = [0,10,22.1,20]

#Delta[2] = Delta[3] -(gF[2] + gF[3])*muB*B

#Delta

5.617233285596307


In [22]:
Delta_ds = []
for mFe in range(-F[0],F[0] + 1):
    for mFg in range(-F[2],F[2] + 1):
        for q in [-1,1]:
            #print(mFg)
            if cg(F,2,mFg,0,mFe,q) != 0:
                for ig in [1,3]:
                    for mFFg in range(-F[ig],F[ig] + 1):
                        for qq in [-1,1]:
                        #print(mFFg)
                            if cg(F,ig,mFFg,0,mFe,qq) != 0:
                                Delta_ds.append(Delta[ig] - gF[2]*mFg*muB*B + gF[ig]*mFFg*muB*B)

In [23]:
Delta_ds.sort()
#Delta_ds

In [24]:
Delta = [0,10,Delta_ds[-3],20]
H0 = H_0(F,Delta)
HI = H_I(F,Omega_p)
H=H0+HI
#make sure Hamiltonian is sparse
#H = H.to("CSR").tidyup(atol=1e-8)
L = -1j*(spre(H)-spost(H))
for i in range(len(c_ops)):
    L += spre(c_ops[i])*spost(c_ops[i].dag()) - 0.5*(spre(c_ops[i].dag()*c_ops[i]) + spost(c_ops[i].dag()*c_ops[i]))
L = L.data_as('ndarray')

In [25]:
np.shape(L)

(3600, 3600)

In [26]:
eigenvalues, eigenvectors = eigs(L,k=10,sigma = 0+0j)

In [27]:
eigenvalues

array([-4.21621376e-18-1.90116306e-17j, -3.68329188e-06+9.50310454e-11j,
       -1.75070546e-05-1.11138984e-08j, -5.99293263e-05+5.07148719e-10j,
       -1.47948071e-04-1.29554888e-07j, -3.02765600e-04-7.23497472e-09j,
       -5.15893113e-04-1.09542735e-07j, -8.18685541e-04-3.33740529e-06j,
       -1.09196591e-03+3.24368993e-06j, -1.83288103e-03+1.72961487e-06j])

In [28]:
eigenvalues.real

array([-4.21621376e-18, -3.68329188e-06, -1.75070546e-05, -5.99293263e-05,
       -1.47948071e-04, -3.02765600e-04, -5.15893113e-04, -8.18685541e-04,
       -1.09196591e-03, -1.83288103e-03])

In [29]:
rhoss = eigenvectors[:,0]
rhos = np.zeros((N,N),dtype='complex64')
for i in range(N):
    for j in range(N):
        rhos[j,i] = rhoss[j+i*N]

rho = Qobj(rhos)
rho = rho/rho.tr()

In [31]:
pops_ = [expect(basis(N,i)*basis(N,i).dag(),rho) for i in range(N)] #steady state rest ion

In [32]:
for j in range(len(pops_)):
    if pops_[j].real>0.01:
        print(j)
        print(pops_[j])

42
(0.989151149012678+3.054595465457055e-11j)
58
(0.010848850988160519-2.5755313246694778e-11j)


In [34]:

sum(pops_)

(1-1.2737091662971659e-17j)

In [35]:
#system parameters
c = 299792458
λ = 646e-9
f = c/λ 
f_1 = 11.2e9
f_2 = 10.5e9

#176Lu+
amu = 1.66053886e-27
hbar = 1.054571817e-34
M = 176*amu

omega0  = 1.4

eta = []

eta2 = 2*np.pi*f/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta2)

eta3 = 2*np.pi*(f-f_1)/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta3)

eta4 = -2*np.pi*(f-f_2)/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta4)

In [36]:
def H1_(F,eta,Omega_p):
    H1=0*basis(N,0)*basis(N,0).dag()
    for mFe in range(-F[0],F[0] + 1):
        for ig in range(1,len(F)):
            for mFg in range(-F[ig],F[ig]+1):
                for q in range(-1,2):
                    H1 += 1j*eta[ig-1]*cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*base(F,0,mFe)*base(F,ig,mFg).dag()
                
    return -1/np.sqrt(2*F[0]+1)*H1

In [37]:
H0_ = H_0(F,Delta)
HI_ = H_I(F,Omega_p) 
    

H0 = H0_ + HI_ 

c=c_ops 

L0 = 0*spre(c[0])*spost(c[0].dag())
for i in range(len(c)):
    L0 += spre(c[i])*spost(c[i].dag()) - 0.5*(spre(c[i].dag()*c[i]) + spost(c[i].dag()*c[i]))

L0 += -1j*(spre(H0) - spost(H0))
    
#L0 = np.array(L0)
L0 = L0.data_as('ndarray')

H_1 = H1_(F,eta,Omega_p)
L_1 = -1j*(spre(H_1) - spost(H_1))

#L_1 = np.array(L_1)
L_1 = L_1.data_as('ndarray')

H1 = H_1.dag()
L1 = -1j*(spre(H1) - spost(H1))

#L1 = np.array(L1)
L1 = L1.data_as('ndarray')

delta=omega0

S3 = -np.matmul(inv(L0-3j*delta*np.eye(N**2)),L1)
S2 = -np.matmul(inv(L0-2j*delta*np.eye(N**2)+ np.matmul(L_1,S3)),L1)
S1 = -np.matmul(inv(L0-1j*delta*np.eye(N**2) + np.matmul(L_1,S2)),L1) 

T_3 = -np.matmul(inv(L0+3j*delta*np.eye(N**2)),L_1)
T_2 = -np.matmul(inv(L0+2j*delta*np.eye(N**2)+ np.matmul(L1,T_3)),L_1)
T_1 = -np.matmul(inv(L0+1j*delta*np.eye(N**2)+ np.matmul(L1,T_2)),L_1)
    
L = np.matmul(L_1,S1) + L0 +np.matmul(L1,T_1)
    
eigenvalues, eigenvectors = eigs(L,k=10,sigma = 0+0j)

In [38]:
eigenvalues

array([ 1.22655457e-17-1.28871030e-17j, -2.23074139e-05-7.08112668e-08j,
       -1.15551818e-04-9.20055190e-08j, -2.64812616e-04-3.22033890e-07j,
       -4.94910815e-04+9.27544359e-07j, -9.85263579e-04-1.18946096e-05j,
       -1.26482089e-03-1.01327236e-05j, -3.16131346e-03+1.18543653e-04j,
       -3.15935598e-03-1.21936845e-04j, -3.28457406e-03+3.29463559e-08j])

In [39]:
rhoss = eigenvectors[:,0]
rhos = np.zeros((N,N),dtype='complex64')
for i in range(N):
    for j in range(N):
        rhos[j,i] = rhoss[j+i*N]

rho = Qobj(rhos)
rho = rho/rho.tr()

In [40]:
pops_1 = [expect(basis(N,i)*basis(N,i).dag(),rho) for i in range(N)] #fictitious lasers

In [41]:
for j in range(len(pops_1)):
    if pops_1[j].real>0.01:
        print(j)
        print(pops_1[j])

40
(0.031146495116539902-4.65687913578372e-11j)
41
(0.012825590110106585-6.235748462729562e-11j)
42
(0.9069062652180293+4.704189815463167e-11j)
58
(0.01107421408573826+3.463003495074446e-11j)


In [43]:
pop_cool = np.array( #average population during cooling
[0.0011792390641969844,
 0.0012107611218760176,
 0.0010255929619416102,
 0.0010022121160483766,
 0.0007659574247022892,
 0.000696084652191965,
 0.0005383331434219447,
 0.0004712277077444033,
 0.00040460206684772424,
 0.0004163657977780548,
 0.00038797658897879163,
 0.0003544372343916331,
 0.0002914329008243498,
 0.00031678219672375566,
 0.0002489600098400819,
 0.00987315390235903,
 0.00633404250971594,
 0.004296719532202632,
 0.0031178868028897545,
 0.0022155466618861653,
 0.0018345555407735443,
 0.001450869969824593,
 0.0013864114384715615,
 0.0012158140680596525,
 0.0011109152512098243,
 0.0008674738902752577,
 0.0008715133846191664,
 0.0006337424080750469,
 0.01634014487933862,
 0.007213378885346397,
 0.011053759191608578,
 0.008358395516808595,
 0.010352943371152062,
 0.009769854447761257,
 0.011501916857259112,
 0.013622762792136357,
 0.01861843724887808,
 0.025926024702848872,
 0.04137904118766621,
 0.07802206103546994,
 0.16669095577568177,
 0.17434586060555118,
 0.2524212362010396,
 0.0065425218828121575,
 0.005306903740025843,
 0.004897942767832944,
 0.0062730300963799845,
 0.0062721006635465745,
 0.006754317209955248,
 0.006246971083598386,
 0.006678739595735177,
 0.004987811175221999,
 0.005180642288455225,
 0.004598258756731275,
 0.006823804749600693,
 0.008004013547505752,
 0.01104637481806433,
 0.009644223951170915,
 0.00824055811259824,
 0.0023664024776241977])

In [44]:
for j in range(len(pop_cool)):
    if pop_cool[j].real>0.1:
        print(j)
        print(pop_cool[j])

40
0.16669095577568177
41
0.17434586060555118
42
0.2524212362010396


In [45]:
#steady state populations of full system
pops_int = np.array([1.64280000e-05, 3.60000000e-07, 9.26460000e-05, 1.01800000e-06,
       3.11240000e-05, 2.36400000e-06, 6.23180000e-05, 2.21600000e-06,
       7.31520000e-05, 7.26000000e-07, 7.74720000e-05, 3.16600000e-06,
       5.26600000e-05, 1.64600000e-06, 2.71580000e-05, 5.76120000e-05,
       3.68000000e-07, 1.06765600e-03, 1.16200000e-06, 2.22650000e-04,
       1.77400000e-06, 6.75956000e-04, 6.14000000e-07, 2.70254000e-04,
       1.10800000e-06, 3.68414000e-04, 7.44000000e-07, 9.46540000e-05,
       3.48760000e-05, 1.26923600e-03, 2.50740000e-05, 8.53514000e-04,
       7.35240000e-05, 8.30900000e-04, 3.50722000e-04, 1.85675200e-03,
       1.02940600e-03, 4.75090400e-03, 1.66055800e-03, 2.00082860e-02,
       1.44802820e-02, 1.23917400e-01, 8.08399232e-01, 1.57016000e-04,
       1.47000000e-06, 2.59278000e-04, 4.05400000e-06, 1.58040000e-04,
       1.41940000e-05, 1.40612000e-04, 2.96160000e-05, 1.65340000e-04,
       2.51500000e-05, 2.92932000e-04, 4.31140000e-05, 8.94508000e-04,
       5.33522000e-04, 3.92357000e-03, 9.27256400e-03, 1.33693000e-03])

In [46]:
pops_ = np.array(pops_)

In [47]:
pops_1 = np.array(pops_1)

In [48]:
from numpy import linalg as LA

In [49]:
LA.norm(pops_int - pops_)

0.22066418850230926

In [50]:
LA.norm(pops_int - pops_1)

0.1502540029626242

In [51]:
LA.norm(pop_cool - pops_)

0.7821171717431635

In [52]:
LA.norm(pop_cool - pops_1)

0.6942256192696029